In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import intake
import dask

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
url = "https://raw.githubusercontent.com/andrewpauling/cmip6hack-so-project/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)

In [52]:
def get_areacello(col):
    """
    Function to find all the models that have areacello in whatever experiment and ensemble member, and 
    combine them into a dictionary of datasets
    """
    cat = col.search(table_id='Ofx', variable_id='areacello', grid_label='gn')
    name_list = cat.unique('source_id')['source_id']['values']
    
    big_dict = dict()
    for model in name_list:
        tmp = cat.search(source_id=model)
        uni_dict = tmp.unique('member_id')['member_id']['values']
        member_id = uni_dict[0]
        tmpout = tmp.search(member_id=member_id)
        uni_dict2 = tmpout.unique('experiment_id')['experiment_id']['values']
        expt_id = uni_dict2[0]
        tmpout2 = tmpout.search(experiment_id=expt_id)
        
        ds_dict = tmpout2.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False},
                                     cdf_kwargs={'chunks': {}, 'decode_times': False})
    
        big_dict.update(ds_dict)
        
    return big_dict


In [53]:
big_dict = get_areacello(col)

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:


In [49]:
big_dict

{'CMIP.CCCma.CanESM5.historical.Ofx.gn': <xarray.Dataset>
 Dimensions:             (i: 360, j: 291, member_id: 1, vertices: 4)
 Coordinates:
   * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
   * j                   (j) int32 0 1 2 3 4 5 6 ... 284 285 286 287 288 289 290
   * member_id           (member_id) <U9 'r10i1p1f1'
 Dimensions without coordinates: vertices
 Data variables:
     latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
     vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
     vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
     longitude           (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
     areacello           (member_id, j, i) float32 dask.array<chunksize=(1, 291, 360), meta=np.ndarray>
 Attributes:
     CCCma_model_hash:            55f484f90aff0e32c5a8e92a42c6b9ae7ffe6224
     CC